# Set up

In [29]:
import json
import os
import aiohttp

import firebase_admin
from firebase_admin import firestore
from google.cloud.firestore import Client
from rich import print
from tqdm import tqdm_notebook
import pyEX as pyex
import yahooquery as yq
import pandas as pd
from dotenv import load_dotenv

load_dotenv()

api_key = os.environ.get("ALPACA_KEY", "")
api_secret = os.environ.get("ALPACA_SECRET", "")


In [2]:

with open("/Users/peter/Projects/socii/app/socii-development-serviceAccountKey.json", "r") as f:
    service_account_info = json.load(f)

credentials = firebase_admin.credentials.Certificate(service_account_info)

# initialize firebase admin
firebase_app = firebase_admin.initialize_app(
    credentials, options=dict(storageBucket="sociiinvest.appspot.com")
)

client: Client = firestore.client()
iex = pyex.Client(api_token=os.environ.get("IEX_TOKEN", ""))


In [3]:
async def get_broker_api_endpoint(endpoint:str, params:dict):
    async with aiohttp.ClientSession(
        auth=aiohttp.BasicAuth(api_key, api_secret)
    ) as session:
        async with session.get(
            os.environ.get("ALPACA_BASE_URL", "") + endpoint + 
            ("?" + "&".join([f"{k}={v}" for k, v in params.items()]) if params else ""),
        ) as response:
            data = await response.json()
            return data


In [4]:
async def write_alpaca_assets_to_file():
    assets = await get_broker_api_endpoint("/v1/assets", {})
    with open("./alpaca/data/assets.json", "w") as f:
        json.dump(assets, f)

await write_alpaca_assets_to_file()

In [5]:
def read_alpaca_assets_from_file():
    with open("./alpaca/data/assets.json", "r") as f:
        return json.load(f)

alpaca_assets = read_alpaca_assets_from_file()

In [6]:
alpaca_assets_df = pd.DataFrame.from_dict(alpaca_assets)

In [7]:
alpaca_assets_df.status.value_counts()

active      9974
inactive    2571
Name: status, dtype: int64

# Disable based on alpaca `status`

In [25]:

def disable_alpaca_assets_which_are_inactive():
    for asset in tqdm(alpaca_assets_df.itertuples()):
        if asset.status == "inactive":
            query = client.collection("tickers").where("alpaca.id", "==", asset.id)
            docs = query.stream()

            for doc in docs:
                print(f"{doc.id} => {doc.to_dict()}")
                client.document(f"tickers/{doc.id}").set(
                    {"isDisabled": True,
                     "alpaca": {
                         "status": 'inactive', 
                         "tradable": asset.tradable,
                         "marginable": asset.marginable,
                         "shortable": asset.shortable,
                         "easy_to_borrow": asset.easy_to_borrow,
                         "fractionable": asset.fractionable
                     }
                    }, merge=True)

In [ ]:
def enable_alpaca_assets_which_are_active():
    for asset in tqdm(alpaca_assets_df.itertuples()):
        if asset.status == "active":
            query = client.collection("tickers").where("alpaca.id", "==", asset.id)
            docs = query.stream()

            for doc in docs:
                print(doc.id)
                client.document(f"tickers/{doc.id}").set(
                    {"isDisabled": False,
                     "alpaca": {
                         "status": 'active', 
                         "tradable": asset.tradable,
                         "marginable": asset.marginable,
                         "shortable": asset.shortable,
                         "easy_to_borrow": asset.easy_to_borrow,
                         "lastUpdated": firestore.SERVER_TIMESTAMP,
                         "fractionable": asset.fractionable
                     }
                    }, merge=True)

## Checking Exchanges

The following shows we only need to query `NASDAQ NYSE AMEX` as the others are either ETFs or inactive

In [38]:
alpaca_assets_df['exchange'].value_counts()

NASDAQ    5287
NYSE      3966
ARCA      2094
BATS       594
AMEX       346
OTC        258
Name: exchange, dtype: int64

In [52]:
alpaca_assets_df.query("exchange == 'NASDAQ' and status == 'active'")

,id,class,exchange,symbol,name,status,tradable,marginable,shortable,easy_to_borrow,fractionable
8,85e18d9a-652b-49f6-87fd-841d571602df,us_equity,NASDAQ,AAON,"AAON, Inc. Common Stock",active,True,True,True,True,False
15,df913064-06d2-40c3-99ad-b91d96465c8b,us_equity,NASDAQ,AAXJ,iShares MSCI All Country Asia ex Japan ETF,active,True,True,True,True,False
21,116e688d-67e2-4a08-a681-461ae5e776aa,us_equity,NASDAQ,ABGI,ABG Acquisition Corp. I Class A Ordinary Shares,active,True,True,True,True,False
22,ddc3a7ce-23a9-4d94-8042-22b6b5fac1f5,us_equity,NASDAQ,ABSI,Absci Corporation Common Stock,active,True,True,False,False,False
23,5dc085fd-240a-41e7-a2f4-8831366fce5e,us_equity,NASDAQ,ABST,Absolute Software Corporation Common Stock,active,True,True,True,True,False
...,...,...,...,...,...,...,...,...,...,...,...
12530,75c9e191-bf26-4eed-891a-b9c862189c96,us_equity,NASDAQ,MBIN,Merchants Bancorp Common Stock,active,True,True,True,True,False
12531,abc8f93b-8f15-4018-b86a-c98b2d1dca4c,us_equity,NASDAQ,MDIA,Mediaco Holding Inc. Class A Common Stock,active,True,True,False,False,False
12532,542013b0-b73c-492b-adf4-5c65e49da576,us_equity,NASDAQ,METC,"Ramaco Resources, Inc. Common Stock",active,True,True,True,True,False
12534,fbf26e5a-aef0-498a-8f70-f5e95e397393,us_equity,NASDAQ,NYMX,Nymox Pharmaceutical Corporation Common Stock ...,active,True,True,False,False,False


# Disabling stocks with do not have `alpaca` field and whose `marketCountry` is not the USA

In [58]:
alpaca_symbols = alpaca_assets_df["symbol"].to_list()

In [ ]:
track = {}
query = client.collection("tickers").where("marketCountry", "!=", "United States of America")
docs = query.stream()

for doc in tqdm(docs):
    data = doc.to_dict()
    if "alpaca" not in data:
        print(f"{doc.id} => {data}")
        if data['tickerSymbol'].split(".")[0] in alpaca_symbols:
            track[doc.id] = data
        client.document(f"tickers/{doc.id}").set({"isDisabled": True, 'lastUpdated': firestore.SERVER_TIMESTAMP, 'alpaca': None }, merge=True)


In [69]:
# write the tracked asset results from `track` into local file
with open("./alpaca/data/alpaca_assets_outside_us.json", "w") as f:
    json.dump(track, f)


In [81]:
for db_asset in track.values():
    stripped_ticker, yahoo_suffix  = db_asset['tickerSymbol'].split(".")
    print(db_asset)
    print(alpaca_assets_df.query("symbol == @stripped_ticker"))
    break
    

ModuleNotFoundError: No module named 'Levenshtein'

# Check for new alpaca stocks not yet added to firestore

## BEWARE THIS WILL CAUSE ALOT OF READS

In [ ]:
# db_assets = {}
# for asset_id in tqdm(alpaca_assets_df["id"]):
#     query = client.collection("tickers").where("alpaca.id", "==", asset_id)
#     for doc in query.stream():
#         db_assets[doc.id] = doc.to_dict()

In [105]:
def stringify_json(d):
    for k, v in d.items():
        if isinstance(v, dict):
            stringify_json(v)
        else:
            if type(v) != str:
                v = str(v)
            d.update({k: v})
    return d


In [104]:
with open("./alpaca/data/firestore_alpaca_assets.json", "w") as f:
    json.dump(stringify_json(db_assets), f)

# Comparing alpaca assets & those found in firestore

In [93]:
alpaca_assets_df.query("exchange in ('NASDAQ', 'NYSE', 'AMEX') and status == 'active'")

,id,class,exchange,symbol,name,status,tradable,marginable,shortable,easy_to_borrow,fractionable
0,73706fa6-f7b8-4919-9381-ff7377a56726,us_equity,NYSE,MCK,McKesson Corporation,active,True,True,True,True,True
2,6999555d-ad35-453f-8037-ea1ea6aa88fc,us_equity,AMEX,GLDG,GoldMining Inc.,active,True,True,False,False,False
3,5ae07a68-48de-4b8e-8105-e50c47689464,us_equity,NYSE,GPC,Genuine Parts Company,active,True,True,True,True,True
4,1bb6a1dd-2542-4f59-a51a-813e4af0abf2,us_equity,AMEX,RCG,RENN Fund Inc.,active,True,True,False,False,False
5,7d0079bc-1597-4355-8c11-0bf51c1284cd,us_equity,NYSE,AAC,Ares Acquisition Corporation,active,True,True,True,True,False
...,...,...,...,...,...,...,...,...,...,...,...
12537,324166e1-e5cc-4aae-99a5-e129132dc542,us_equity,NASDAQ,PEBK,"Peoples Bancorp of North Carolina, Inc. Common...",active,True,True,True,True,False
12539,10f7d07f-0811-42bf-a780-b4d4cb53eb91,us_equity,NYSE,SPGI,S&P Global Inc.,active,True,True,True,True,True
12540,aee4e453-1836-4e9c-9f2c-5d1b4f624076,us_equity,NYSE,SPXC,SPX Corporation,active,True,True,True,True,False
12542,3f3e0ff9-599f-4fec-8842-6bc53f5129a1,us_equity,NYSE,WMT,Walmart Inc.,active,True,True,True,True,True


In [110]:
with open("./alpaca/data/firestore_alpaca_assets.json", "r") as f:
    firestore_alpaca_assets_df = pd.DataFrame.from_dict(json.load(f)).T

In [123]:
firestore_alpaca_ids = firestore_alpaca_assets_df.alpaca.map(lambda x: x["id"]).to_list()
with pd.option_context("display.max_rows", None):
    display(alpaca_assets_df.query('id not in @firestore_alpaca_ids and exchange in ("NASDAQ", "NYSE", "AMEX") and status == "active"').reset_index(drop=True))

# These are mostly ETFs so no need to rush to get them on the platform. There are some stocks and to find their ISINs we need to scrape them
# For example we could scrape https://stockanalysis.com/stocks/bbwi/company/ for Bath and Body Works (BBWI)

,id,class,exchange,symbol,name,status,tradable,marginable,shortable,easy_to_borrow,fractionable
0,df913064-06d2-40c3-99ad-b91d96465c8b,us_equity,NASDAQ,AAXJ,iShares MSCI All Country Asia ex Japan ETF,active,True,True,True,True,False
1,ddc3a7ce-23a9-4d94-8042-22b6b5fac1f5,us_equity,NASDAQ,ABSI,Absci Corporation Common Stock,active,True,True,False,False,False
2,bd96cb48-195b-45fa-94b2-20841a06e9b7,us_equity,NASDAQ,ABVC,"ABVC Biopharma, Inc. Common Stock",active,True,True,False,False,False
3,2f6c684e-6f06-4db7-8776-bac9a7093601,us_equity,NASDAQ,ACWI,iShares MSCI ACWI ETF,active,True,True,True,True,True
4,ca8d2d9c-d975-45d5-87fe-ea00cd67d766,us_equity,NYSE,ML,MoneyLion Inc.,active,True,True,False,False,True
5,837d28de-c351-423f-b4f2-ae4365c3f433,us_equity,NASDAQ,ADGI,"Adagio Therapeutics, Inc. Common Stock",active,True,True,True,True,False
6,1d19147c-d56e-4055-8578-ba54ccde6dd2,us_equity,NASDAQ,ADOCR,Edoc Acquisition Corp. Right,active,True,True,True,True,False
7,1b212a11-f052-4ee3-826d-a676474e3615,us_equity,NASDAQ,ADRE,Invesco BLDRS Emerging Markets 50 ADR Index Fund,active,True,True,True,True,False
8,22d77b19-025f-4c9d-82c7-01c982c9f425,us_equity,NASDAQ,ADV,Advantage Solutions Inc. Class A Common Stock,active,True,True,True,True,False
9,a0e4339c-2522-4025-a960-d8fd15a6726b,us_equity,NYSE,ADX,"Adams Diversified Equity Fund, Inc",active,True,True,False,False,False
